# 1. Pobranie sciezek do kanalow obrazow Sentinel-2

In [1]:
from osgeo import gdal, gdalconst
import os
import numpy as np

In [2]:
def get_bands(bands_filepath, band_nrs):
    """
    Funkcja znajduje pliki i przypisuje sciezki do nich jako wartosci w wynikowym slowniku
    parametry:  bands_filepath - sciezka do folderu z rozdzielczosciami przestrzennymi (IMG_DATA),
                bands_nrs - lista z szukanymi kanalami
    zwraca: bands_dict - slownik, gdzie klucz: oznaczenie kanalu, np. 'B02', a wartosc: sciezka do pliku
    """
    bands_dict ={}
    files = os.listdir(bands_filepath)
    for file in files:
        band_nr = file[-7:-4]
        if band_nr in band_nrs:
            bands_dict[band_nr] = "{}\{}".format(bands_filepath, file)
    return bands_dict

In [3]:
def resample_bands_to_10m_sr(band_path, factor, output_path):
    input = gdal.Open(band_path, gdalconst.GA_ReadOnly)
    inputProj = input.GetProjection()
    inputTrans = (input.GetGeoTransform()[0], 10.0, input.GetGeoTransform()[2], 
              input.GetGeoTransform()[3], input.GetGeoTransform()[4], -10.0)
    x = input.RasterXSize*factor
    y = input.RasterYSize*factor
    bandreference = input.GetRasterBand(1)   

    driver= gdal.GetDriverByName('GTiff')
    output = driver.Create(output_path,x,y,1,bandreference.DataType)
    output.SetGeoTransform(inputTrans)
    output.SetProjection(inputProj)

    gdal.ReprojectImage(input,output,inputProj,inputProj,gdalconst.GRA_CubicSpline)

    del output

In [4]:
images_path = {"20200407" : r"D:\Sentinele\S2B_MSIL1C_20200407T095029_N0209_R079_T34UDE_20200407T115232\S2B_MSIL1C_20200407T095029_N0209_R079_T34UDE_20200407T115232.SAFE\GRANULE\L1C_T34UDE_A016121_20200407T095026\IMG_DATA",
              "20200408": r"D:\Sentinele\S2A_MSIL1C_20200408T101021_N0209_R022_T33UWV_20200408T153254\S2A_MSIL1C_20200408T101021_N0209_R022_T33UWV_20200408T153254.SAFE\GRANULE\L1C_T33UWV_A025044_20200408T101022\IMG_DATA",
              "20200409": r"D:\Sentinele\S2A_MSIL1C_20200409T094031_N0209_R036_T34UEC_20200409T114209\S2A_MSIL1C_20200409T094031_N0209_R036_T34UEC_20200409T114209.SAFE\GRANULE\L1C_T34UEC_A025058_20200409T094554\IMG_DATA"}

In [5]:
images_workspace = {"20200407": r"D:\Sentinele\resampling_20200407",
                   "20200408": r"D:\Sentinele\resampling_20200408",
                   "20200409": r"D:\Sentinele\resampling_20200409"}

In [6]:
bands_nrs = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B10", "B11", "B12"]

In [7]:
image_date = "20200408"

In [ ]:
bands_files = get_bands(images_path[image_date], bands_nrs)
    
for key, value in bands_files.items():
    output_path = "{}\{}{}".format(images_workspace[image_date], key, ".tif")
    if key in ("B05", "B06", "B07", "B8A", "B11", "B12"):
        resample_bands_to_10m_sr(value, 2, output_path)
        bands_files[key] = output_path
    elif key in ("B01", "B09", "B10"):
        resample_bands_to_10m_sr(value, 6, output_path)
        bands_files[key] = output_path

# 2. Wczytanie obrazow do numpy array 

In [9]:
data = []
for i in bands_nrs:
    data.append(gdal.Open(bands_files[i]).ReadAsArray())

In [10]:
for d in data:
    print(d.shape)

(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)
(10980, 10980)


In [11]:
data = np.stack(data, axis=-1).astype('float32')

In [12]:
data.shape

(10980, 10980, 13)

In [13]:
remove_idx = 10
data = np.concatenate(
    ( data[:,:,0:remove_idx],
      data[:, :, remove_idx+1:]),
    axis=2)
data.shape

(10980, 10980, 12)

# 3. Normalizacja

In [14]:
data = np.reshape(data, (1, 10980, 10980, 12))

In [15]:
data.shape

(1, 10980, 10980, 12)

In [16]:
def normalize(x):
    x_means = np.zeros(x.shape[0])
    x_stds=np.zeros(x.shape[0])
    
    for i in range(x.shape[0]):
        x_means[i]=x[i,:].mean()
        x_stds[i]=x[i,:].std()
    x_stds+=1e-5
    for i in range(x.shape[0]):
        x[i]-=x_means[i]
        x[i]/=x_stds[i]

In [17]:
normalize(data)

# 4. Predykcja

In [18]:
from keras.models import load_model
model = load_model(r"C:\Users\msokolowska\INZ\mery_final_inz\wetransfer-febc02\saved_model_12dims.h5")

Using TensorFlow backend.


In [19]:
classes = {0: 'Pasture',
 1: 'Industrial',
 2: 'River',
 3: 'Residential',
 4: 'HerbaceousVegetation',
 5: 'PermanentCrop',
 6: 'Highway',
 7: 'SeaLake',
 8: 'AnnualCrop',
 9: 'Forest'}

In [20]:
raster = gdal.Open(bands_files['B02'])
#ulx - Upper left corner x; uly - Upper left corner y
ulx, xres, xskew, uly, yskew, yres  = raster.GetGeoTransform()
#Lower right corner x
lrx = ulx + (raster.RasterXSize * xres)
#Lower right corner y
lry = uly + (raster.RasterYSize * yres)
print(bands_files['B02'])

D:\Sentinele\S2A_MSIL1C_20200408T101021_N0209_R022_T33UWV_20200408T153254\S2A_MSIL1C_20200408T101021_N0209_R022_T33UWV_20200408T153254.SAFE\GRANULE\L1C_T33UWV_A025044_20200408T101022\IMG_DATA\T33UWV_20200408T101021_B02.jp2


4.1. Odwolanie do ArcGIS Feature Class

In [21]:
import arcpy

In [ ]:
#Stworzenie klasy obiektow
proj = raster.GetProjection()
gdb_path = r"C:\Users\msokolowska\INZ\INZ\Default.gdb"
name = "obraz{}".format(image_date)
result = arcpy.CreateFeatureclass_management(gdb_path, name, "POLYGON", "#", "#", "#", proj)
arcpy.AddField_management("{}\{}".format(gdb_path, name), "LandUseorCover", "TEXT")
fc = result[0]

In [24]:
#Odwolanie do instejacej klasy obiektow
arcpy.env.workspace = r"C:\Users\msokolowska\INZ\INZ\Default.gdb"
fc = arcpy.ListFeatureClasses()[5]
print(fc)

obraz20200408ok


Pierwsze okno 64x64 piksele

In [31]:
#wspolrzedne terenowe
rasterxsize = raster.RasterXSize
xupleft = ulx
yupleft = uly

xupright = xupleft+64*xres
yupright = uly

xdownright = xupright
ydownright = (yupright + 64*yres)

xdownleft = xupleft
ydownleft = ydownright


#wspolrzedne obrazu
xstart = 0
xend = 64
ystart =0
yend = 64

In [32]:
rasterxsize = 1000
lry = yupleft + rasterxsize*yres

In [38]:
def _get_window(big_picture, x, y, w, h):  
    return big_picture[y:y + h, x:x + w, :].reshape((1, h, w, -1))  

def window_iter(big_picture, xstart=0, ystart=0, window_size=64, stepx=64, stepy=64):  
    """  
    Args:  
        big_picture: [HWC]  
        nwindows: if -1, all windows  
    Returns:  
        tuple: (window, x0, y0), where x0, y0 - upper left coordinate in big_picture  
    """  
    bpsh = big_picture.shape  
    x = xstart  
    y = ystart  
    while y + window_size < bpsh[0]:  
        yield _get_window(big_picture, x, y, window_size, window_size), x, y  
        x = x + stepx if x + stepx + window_size < bpsh[1] else 0  
        if x == 0:  
            y += stepy

In [39]:
for window, x, y in window_iter(data[0, :1000, :1000, :]):
    pred = model.predict(window)
    pred_word = classes[np.argmax(pred[0])]
    print(pred_word)

Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Residential
Resi

In [33]:
while round(ydownright,4) >= round(lry,4):
    coordinates = [(xupleft, yupleft),
              (xupright, yupright),
              (xdownright, ydownright),
              (xdownleft, ydownleft)]
    
    #tutaj jakos polaczyc tensor data z wspolrzednymi przestrzennymi
    #data_part = data[:, xstart:xend, ystart:yend, :]
    data_part = data[:, ystart:yend, xstart:xend, :]
    pred = model.predict(data_part)
    pred_word = classes[np.argmax(pred[0])]
    
    with arcpy.da.InsertCursor(fc, ["LandUseorCover", 'SHAPE@']) as cursor:
        cursor.insertRow([pred_word, coordinates])
    
    if xupright+64*xres < ulx+rasterxsize*xres:
        xupleft = xupleft+64*xres
        xupright = xupright+64*xres
        xdownleft = xupleft
        xdownright = xupright
        
        xstart = xstart+64
        xend = xend+64
    else:
        xupleft = ulx
        yupleft = ydownleft
        xupright = xupleft+64*xres
        yupright = yupleft
        xdownright = xupright
        ydownright = (yupright + 64*yres)
        xdownleft = xupleft
        ydownleft = ydownright
        
        xstart = 0
        xend =64
        ystart = ystart+64
        yend = yend+64

In [36]:
pred

array([[8.0139354e-15, 3.1371790e-14, 2.6259073e-12, 9.8394591e-01,
        1.6053861e-02, 1.8773974e-18, 6.8323857e-08, 3.0817193e-19,
        1.3272653e-15, 8.8971852e-08]], dtype=float32)